In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer

from sklearn.ensemble import RandomForestClassifier

import xgboost as xgb

### Data preparation

In [2]:
data = 'https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-06-trees/CreditScoring.csv'
df = pd.read_csv(data)

In [3]:
df.columns = df.columns.str.lower()

status_values = {
    1: 'ok',
    2: 'default',
    0: 'unk'
}

df.status = df.status.map(status_values)

home_values = {
    1: 'rent',
    2: 'owner',
    3: 'private',
    4: 'ignore',
    5: 'parents',
    6: 'other',
    0: 'unk'
}

df.home = df.home.map(home_values)

marital_values = {
    1: 'single',
    2: 'married',
    3: 'widow',
    4: 'separated',
    5: 'divorced',
    0: 'unk'
}

df.marital = df.marital.map(marital_values)

records_values = {
    1: 'no',
    2: 'yes',
    0: 'unk'
}

df.records = df.records.map(records_values)

job_values = {
    1: 'fixed',
    2: 'partime',
    3: 'freelance',
    4: 'others',
    0: 'unk'
}

df.job = df.job.map(job_values)

for c in ['income', 'assets', 'debt']:
    df[c] = df[c].replace(to_replace=99999999, value=np.nan)

df = df[df.status != 'unk'].reset_index(drop=True)

In [4]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=11)

df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

y_train = (df_train.status == 'default').astype('int').values
y_test = (df_test.status == 'default').astype('int').values

del df_train['status']
del df_test['status']

In [5]:
dv = DictVectorizer(sparse=False)

train_dicts = df_train.fillna(0).to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

test_dicts = df_test.fillna(0).to_dict(orient='records')
X_test = dv.transform(test_dicts)

### Random forest

In [6]:
rf = RandomForestClassifier(n_estimators=200,
                            max_depth=10,
                            min_samples_leaf=3,
                            random_state=1)
rf.fit(X_train, y_train)

RandomForestClassifier(max_depth=10, min_samples_leaf=3, n_estimators=200,
                       random_state=1)

### XGBoost

Note:

We removed feature names

It was 

```python
features = dv.get_feature_names_out()
dtrain = xgb.DMatrix(X_train, label=y_train, feature_names=features)
```

Now it's

```python
dtrain = xgb.DMatrix(X_train, label=y_train)
```

In [7]:
dtrain = xgb.DMatrix(X_train, label=y_train)

In [8]:
xgb_params = {
    'eta': 0.1, 
    'max_depth': 3,
    'min_child_weight': 1,

    'objective': 'binary:logistic',
    'eval_metric': 'auc',

    'nthread': 8,
    'seed': 1,
    'verbosity': 1,
}

model = xgb.train(xgb_params, dtrain, num_boost_round=175)

### BentoML

In [9]:
import bentoml

In [10]:
bentoml.xgboost.save_model(
    'credit_risk_model',
    model,
    custom_objects={
        'dictVectorizer': dv
    })

Model(tag="credit_risk_model:6l3xq4sr7cp57jgr", path="/Users/dg/bentoml/models/credit_risk_model/6l3xq4sr7cp57jgr/")

Test

In [11]:
import json

In [12]:
request = df_test.iloc[0].to_dict()
print(json.dumps(request, indent=2))

{
  "seniority": 3,
  "home": "owner",
  "time": 36,
  "age": 26,
  "marital": "single",
  "records": "no",
  "job": "freelance",
  "expenses": 35,
  "income": 0.0,
  "assets": 60000.0,
  "debt": 3000.0,
  "amount": 800,
  "price": 1000
}


In [ ]:
'''
Email from your manager

Good morning recruit! It's good to have you here! I have an assignment for you. I have a data scientist that's built a credit risk model in a jupyter notebook. I need you to run the notebook and save the model with BentoML and see how big the model is. If it's greater than a certain size, I'm going to have to request additional resources from our infra team. Please let me know how big it is.

Thanks,

Mr McManager
'''

In [13]:
'''
Question 1

    Install BentoML
    What's the version of BentoML you installed?
    Use --version to find out
'''
!bentoml --version

bentoml, version 1.0.7


In [17]:
'''
Question 2

Run the notebook which contains random forest model from module 6 i.e previous module and save the model with BentoML. To make it easier for you we have prepared this notebook.

How big approximately is the saved BentoML model? Size can slightly vary depending on your local development environment. Choose the size closest to your model.

    924kb
    724kb
    114kb
    8kb
'''
!ls -hal /Users/dg/bentoml/models/credit_risk_model/6l3xq4sr7cp57jgr

total 416
drwxr-xr-x  5 dg  staff   160B Oct 22 13:01 .
drwxr-xr-x  4 dg  staff   128B Oct 22 13:01 ..
-rw-r--r--  1 dg  staff   716B Oct 22 13:01 custom_objects.pkl
-rw-r--r--  1 dg  staff   366B Oct 22 13:01 model.yaml
-rw-r--r--  1 dg  staff   197K Oct 22 13:01 saved_model.ubj


In [18]:
'''
Another email from your manager

Great job recruit! Looks like I won't be having to go back to the procurement team. Thanks for the information.

However, I just got word from one of the teams that's using one of our ML services and they're saying our service is "broken" and their trying to blame our model. I looked at the data their sending and it's completely bogus. I don't want them to send bad data to us and blame us for our models. Could you write a pydantic schema for the data that they should be sending? That way next time it will tell them it's their data that's bad and not our model.

Thanks,

Mr McManager
'''

'\nAnother email from your manager\n\nGreat job recruit! Looks like I won\'t be having to go back to the procurement team. Thanks for the information.\n\nHowever, I just got word from one of the teams that\'s using one of our ML services and they\'re saying our service is "broken" and their trying to blame our model. I looked at the data their sending and it\'s completely bogus. I don\'t want them to send bad data to us and blame us for our models. Could you write a pydantic schema for the data that they should be sending? That way next time it will tell them it\'s their data that\'s bad and not our model.\n\nThanks,\n\nMr McManager\n'

In [21]:
'''
Question 3

Say you have the following data that you're sending to your service:

{
  "name": "Tim",
  "age": 37,
  "country": "US",
  "rating": 3.14
}

What would the pydantic class look like? You can name the class UserProfile.
'''
from pydantic import BaseModel
class CreditApplication(BaseModel):
    seniority: int
    home: str
    time: int
    age: int
    marital: str
    records: str
    job: str
    expenses: int
    income: float
    assets: float
    debt: float
    amount: int
    price: int

In [22]:
'''
Email from your CEO

Good morning! I hear you're the one to go to if I need something done well! We've got a new model that a big client needs deployed ASAP. I need you to build a service with it and test it against the old model and make sure that it performs better, otherwise we're going to lose this client. All our hopes are with you!

Thanks,

CEO of Acme Corp
'''

"\nEmail from your CEO\n\nGood morning! I hear you're the one to go to if I need something done well! We've got a new model that a big client needs deployed ASAP. I need you to build a service with it and test it against the old model and make sure that it performs better, otherwise we're going to lose this client. All our hopes are with you!\n\nThanks,\n\nCEO of Acme Corp\n"

In [27]:
'''
Question 4

We've prepared a model for you that you can import using:

curl -O https://s3.us-west-2.amazonaws.com/bentoml.com/mlzoomcamp/coolmodel.bentomodel
bentoml models import coolmodel.bentomodel

What version of scikit-learn was this model trained with?

    1.1.1
    1.1.2
    1.1.3
    1.1.4
    1.1.5
'''
#!curl -O https://s3.us-west-2.amazonaws.com/bentoml.com/mlzoomcamp/coolmodel.bentomodel
#!bentoml models import coolmodel.bentomodel
#!bentoml models get coolmodel.bentomodel
!bentoml models get mlzoomcamp_homework

name: mlzoomcamp_homework                                                       
version: qtzdz3slg6mwwdu5                                                       
module: bentoml.sklearn                                                         
labels: {}                                                                      
options: {}                                                                     
metadata: {}                                                                    
context:                                                                        
  framework_name: sklearn                                                       
  framework_versions:                                                           
    scikit-learn: 1.1.1                                                         
  bentoml_version: 1.0.7                                                        
  python_version: 3.9.12                                                        
signatures:                 

In [33]:
'''
Question 5

Create a bento out of this scikit-learn model. The output type for this endpoint should be NumpyNdarray()

Send this array to the Bento:

[[6.4,3.5,4.5,1.2]]

You can use curl or the Swagger UI. What value does it return?

    0
    1
    2
    3

(Make sure your environment has Scikit-Learn installed)
'''
!curl -X 'POST' 'http://0.0.0.0:3000/return_output' -H 'accept: application/json' -H 'Content-Type: application/json' -d '[[6.4,3.5,4.5,1.2]]'

[1]

In [35]:
'''
Question 6

Ensure to serve your bento with --production for this question

Install locust using:

pip install locust

Use the following locust file: locustfile.py

Ensure that it is pointed at your bento's endpoint (In case you didn't name your endpoint "classify")

Configure 100 users with ramp time of 10 users per second. Click "Start Swarming" and ensure that it is working.

Now download a second model with this command:

curl -O https://s3.us-west-2.amazonaws.com/bentoml.com/mlzoomcamp/coolmodel2.bentomodel

Or you can download with this link as well: https://s3.us-west-2.amazonaws.com/bentoml.com/mlzoomcamp/coolmodel2.bentomodel

Now import the model:

bentoml models import coolmodel2.bentomodel

Update your bento's runner tag and test with both models. Which model allows more traffic (more throughput) as you ramp up the traffic?

Hint 1: Remember to turn off and turn on your bento service between changing the model tag. Use Ctl-C to close the service in between trials.

Hint 2: Increase the number of concurrent users to see which one has higher throughput

Which model has better performance at higher volumes?

    The first model
    The second model <- this one!
'''

#!locust -H http://localhost:3000
#!curl -O https://s3.us-west-2.amazonaws.com/bentoml.com/mlzoomcamp/coolmodel2.bentomodel
#!bentoml models import coolmodel2.bentomodel
!bentoml models get mlzoomcamp_homework

name: mlzoomcamp_homework                                                       
version: jsi67fslz6txydu5                                                       
module: bentoml.sklearn                                                         
labels: {}                                                                      
options: {}                                                                     
metadata: {}                                                                    
context:                                                                        
  framework_name: sklearn                                                       
  framework_versions:                                                           
    scikit-learn: 1.1.1                                                         
  bentoml_version: 1.0.7                                                        
  python_version: 3.9.12                                                        
signatures:                 

In [30]:
'''
Email from marketing

Hello ML person! I hope this email finds you well. I've heard there's this cool new ML model called Stable Diffusion. I hear if you give it a description of a picture it will generate an image. We need a new company logo and I want it to be fierce but also cool, think you could help out?

Thanks,

Mike Marketer
'''

"\nEmail from marketing\n\nHello ML person! I hope this email finds you well. I've heard there's this cool new ML model called Stable Diffusion. I hear if you give it a description of a picture it will generate an image. We need a new company logo and I want it to be fierce but also cool, think you could help out?\n\nThanks,\n\nMike Marketer\n"

In [32]:
'''
Question 7 (optional)

Go to this Bento deployment of Stable Diffusion: http://54.176.205.174/ (or deploy it yourself)

Use the txt2image endpoint and update the prompt to: "A cartoon dragon with sunglasses". Don't change the seed, it should be 0 by default

What is the resulting image?
#1

#2

#3

#4
'''

'\nQuestion 7 (optional)\n\nGo to this Bento deployment of Stable Diffusion: http://54.176.205.174/ (or deploy it yourself)\n\nUse the txt2image endpoint and update the prompt to: "A cartoon dragon with sunglasses". Don\'t change the seed, it should be 0 by default\n\nWhat is the resulting image?\n#1\n\n#2\n\n#3\n\n#4\n'